In [ ]:
#imports all necessary packages
from google.colab import files #for google colab
import pandas as pd
import io
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#started with Sample_results sheet
uploaded = files.upload()
for files in uploaded.keys():
      df_sample_results = pd.read_excel(io.BytesIO(uploaded[files]))
      print(f"DataFrame from {files}:")

Saving cleaned_sample_results.xlsx to cleaned_sample_results (12).xlsx
DataFrame from cleaned_sample_results (12).xlsx:


Exploraiton of Sample Results

In [ ]:
df_sample_results.head()

,Name,Workshop_Date,Email,Phone_Number,Resources_List,Resources_Count
0,Tanya B,2025-03-15 00:00:00,ba...23@gmail.com,34...33,"['Cambio Labs', 'REES Go-getters', 'REES Home-...",8
1,Kd C,2025-03-15 00:00:00,kd...52@gmail.com,51...06,['Just the common BKPL Resources'],1
2,Lisa L,2025-03-15 00:00:00,fl...et@verizon.net,64...54,"['Cambio Labs', 'REES Go-getters', 'REES Home-...",8
3,Jasiah V,2025-03-15 00:00:00,Ja...ga2@gmail.com,92...10,"['Cambio Labs', 'REES Dreamers', 'SBS FastTrac...",4
4,Renee B,2025-03-15 00:00:00,dr...22@hotmail.com,64...52,"['Cambio Labs', 'REES Dreamers', 'REES Busines...",6


# Structural engineering of Sample Results

In [ ]:
#fix the 'just the common BKPL Resources column'

# replacement list (ensuring all resource names are enclosed in quotes)
bkpl_replacement_list = '["BKPL Career & Resume Help", "BKPL Business Coaching", "BKPL Business & Career Center", "BKPL PowerUP Competition", "BKPL BKLYN Fashion Academy"]'

df_sample_results['Resources_List'] = df_sample_results['Resources_List'].replace(
    "['Just the common BKPL Resources']",
    bkpl_replacement_list
)


In [ ]:
# Convert the string representation of the Resources_List into an actual Python list
import ast

df_sample_results['Resources_List'] = df_sample_results['Resources_List'].apply(ast.literal_eval)

# Check a row to confirm the list conversion
print(type(df_sample_results.loc[0, 'Resources_List']))

<class 'list'>


CREATING A UNIQUE IDENTIFIER

In [ ]:
# Make a copy before dropping unnecessary columns
df_sample_resultscopy = df_sample_results.copy()

In [ ]:
results_feature_drop=['Name', 'Workshop_Date', 'Phone_Number']
df_sample_results.drop(columns= results_feature_drop, inplace= True)

In [ ]:
df_entrepreneurs= df_sample_results[['Email']].drop_duplicates().rename(columns={'Email': 'Entrepreneur_Email'})  #removes duplicates and rename email column

In [ ]:
#reimport file package to prevent overwriting
#upload program
from google.colab import files as program_files
import io
import pandas as pd

In [ ]:
uploaded = program_files.upload()
for program_files in uploaded.keys():
      df_programs = pd.read_excel(io.BytesIO(uploaded[program_files]))
      print(f"DataFrame from {df_programs}:")

Saving cleaned_programs_data.xlsx to cleaned_programs_data (6).xlsx
DataFrame from                                          program_name  \
0                                              Cambio   
1                                       REES Dreamers   
2                                      REES Idealists   
3                      REES Underground Entrepreneurs   
4                                     REES Go-getters   
5                                      REES Champions   
6                            REES Home-Based Business   
7                  REES Business Development Resource   
8                                                SSTB   
9                          Boss Up Vetern competition   
10                            SBS FastTrac NewVenture   
11  SBS FastTrac® NewVenture™ for the Female Entre...   
12                      SBS FastTrac® NewVenture™ 50+   
13                         SBS FastTrac® TechVenture™   
14                       SBS FastTrac® GrowthVenture™   
15   

In [ ]:
from google.colab import files as program2_files
import io
import pandas as pd

In [ ]:
uploaded = program2_files.upload()
for program2_files in uploaded.keys():
      df_gen_resources = pd.read_excel(io.BytesIO(uploaded[program2_files]))
      print(f"DataFrame from {program2_files}:")

Saving cleaned_resources_for_all_data.xlsx to cleaned_resources_for_all_data (4).xlsx
DataFrame from cleaned_resources_for_all_data (4).xlsx:


Remove unnecessary columns

In [ ]:
#see the results
df_sample_results.head()

,Email,Resources_List,Resources_Count
0,ba...23@gmail.com,"[Cambio Labs, REES Go-getters, REES Home-Based...",8
1,kd...52@gmail.com,"[BKPL Career & Resume Help, BKPL Business Coac...",1
2,fl...et@verizon.net,"[Cambio Labs, REES Go-getters, REES Home-Based...",8
3,Ja...ga2@gmail.com,"[Cambio Labs, REES Dreamers, SBS FastTrac NewV...",4
4,dr...22@hotmail.com,"[Cambio Labs, REES Dreamers, REES Business Dev...",6


In [ ]:
all_resources_from_sample = df_sample_results['Resources_List'].explode().unique()
master_resource_list = set(all_resources_from_sample).union(
    set(df_programs['program_name'].unique()),
    set(df_gen_resources['resource_name'].unique())
)
master_df_resources = pd.DataFrame(list(master_resource_list), columns=['Resource_Name'])

# --- 4. PERFORM CROSS-JOIN (U x R) ---
df_entrepreneurs['key'] = 1
master_df_resources['key'] = 1
df_training_set = pd.merge(df_entrepreneurs, master_df_resources, on='key').drop('key', axis=1)

# --- 5. CREATE TARGET VARIABLE (is_recommended) ---
# Create a temporary table of known positive recommendations (Y=1)
df_positives = df_sample_results[['Email', 'Resources_List']].explode('Resources_List')
df_positives.rename(columns={'Email': 'Entrepreneur_Email', 'Resources_List': 'Resource_Name'}, inplace=True)

# Merge the positives onto the skeleton and fill the rest with 0
df_training_set = pd.merge(
    df_training_set,
    df_positives.assign(is_recommended=1),
    on=['Entrepreneur_Email', 'Resource_Name'],
    how='left'
)
df_training_set['is_recommended'] = df_training_set['is_recommended'].fillna(0).astype(int)

# --- 6. SAVE SKELETON ---
df_training_set.to_csv('df_training_set_skeleton.csv', index=False)
print("Master Training Set Skeleton created and saved as 'df_training_set_skeleton.csv'")
print(df_training_set.head())

Master Training Set Skeleton created and saved as 'df_training_set_skeleton.csv'
  Entrepreneur_Email                              Resource_Name  \
0  ba...23@gmail.com                            REES Go-getters   
1  ba...23@gmail.com             REES Underground Entrepreneurs   
2  ba...23@gmail.com  SBS FastTrac® GrowthVenture™ for Veterans   
3  ba...23@gmail.com                   BKPL PowerUP Competition   
4  ba...23@gmail.com                       BKPL Ask a Librarian   

   is_recommended  
0               1  
1               0  
2               0  
3               0  
4               0  


# Regular Feature engineering of the hot encoded column

In [ ]:
#reimport file package to prevent overwriting
from google.colab import files as gcolab_files
import io
import pandas as pd

In [ ]:
uploaded = gcolab_files.upload()
for gcolab_files in uploaded.keys():
      df_survey_results = pd.read_excel(io.BytesIO(uploaded[gcolab_files]))
      print(f"DataFrame from {gcolab_files}:")

Saving df_final_processed.xlsx to df_final_processed (7).xlsx
DataFrame from df_final_processed (7).xlsx:


Exploration

In [ ]:
df_survey_results.head()

,Email Address,Timestamp,What is your full name?,Which age group are you in?,What is your level of interest in entrepreneurship or being a business owner?_scaled,What is your level of interest in social entrepreneurship (starting a business that can also have a social or environmental impact)?_scaled,What is your level of interest in cooperative business (starting a business that is owned by various community members that have votes and a share of profits)?_scaled,What borough of NYC do you live in?_Brooklyn,What borough of NYC do you live in?_Manhattan,What borough of NYC do you live in?_Queens,...,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_No,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_No,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_No barriers.,"Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_Re-entering the workforce, personal credit score, and money to start business.",Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_Scheduling,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_Single parent trying to figure it out,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_Timing & will. Depression makes staying on track difficult.,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_Well I don’t have a tablet computer or laptop top for my online classes so there maybe interruptions during zoom because I only have my cell phone to work with,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_nan,Do you have any major barriers to participating in entrepreneurship programs that you want to tell us about?_no
0,ba...3@gmail.com,2025-03-15 16:16:26.778,Tanya B,49-64,1.000000,1.000000,0.00,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,kd...2@gmail.com,2025-03-15 16:16:45.279,Kd C,16-24,0.333333,0.000000,0.25,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,fl...t@verizon.net,2025-03-15 16:38:49.889,Lisa L,49-64,0.666667,0.666667,0.00,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,ja...2@gmail.com,2025-03-15 16:41:12.332,Jasiah V,16-24,0.666667,0.666667,0.75,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,dr...2@hotmail.com,2025-03-15 16:59:41.896,Renee B,34-48,1.000000,0.333333,0.50,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
#make a copy of the dataframe before performign any complex procedures
df_user_featurescopy = df_survey_results.copy()

In [ ]:
df_survey_results.shape

(36, 452)

Column renaming

Im using Ruhe's sheet, which has long column name, I think the name changed back to its original size

In [ ]:
list(df_user_featurescopy.columns)

['Email Address',
 'Timestamp',
 'What is your full name?',
 'Which age group are you in?',
 'What is your level of interest in entrepreneurship or being a business owner?_scaled',
 'What is your level of interest in social entrepreneurship (starting a business that can also have a social or environmental impact)?_scaled',
 'What is your level of interest in cooperative business (starting a business that is owned by various community members that have votes and a share of profits)?_scaled',
 'What borough of NYC do you live in?_Brooklyn',
 'What borough of NYC do you live in?_Manhattan',
 'What borough of NYC do you live in?_Queens',
 'What borough of NYC do you live in?_the Bronx',
 'What is your housing status?_Cityfheps/Rent',
 'What is your housing status?_Currently staying but do not officially live in NYCHA',
 'What is your housing status?_Housing',
 'What is your housing status?_I am a Section 8 Voucher holder',
 "What is your housing status?_I don't live in NYHCA or am not a Se

In [ ]:
import re

def final_guaranteed_shortener(name):
    # 1. Start with a clean, lowercase name
    name = str(name).lower()

    # --- TARGETED QUESTION PREFIX STRIPPING ---

    # ID/Standard Columns (Keep these short and clean)
    name = name.replace('email address', 'email')
    name = name.replace('what is your full name?', 'full_name')
    name = name.replace('which age group are you in?', 'age_group')

    # Interest Scales
    name = name.replace('what is your level of interest in entrepreneurship or being a business owner?_scaled', 'interest_entr_s')
    name = name.replace('what is your level of interest in social entrepreneurship (starting a business that can also have a social or environmental impact)?_scaled', 'interest_social_s')
    name = name.replace('what is your level of interest in cooperative business (starting a business that is owned by various community members that have votes and a share of profits)?_scaled', 'interest_coop_s')

    # OHE Features - Strip the entire long question prefix
    name = name.replace('what borough of nyc do you live in?', 'borough_')
    name = name.replace('what is your housing status?', 'housing_')
    name = name.replace('if you live in nycha, what housing development or building do you live in?', 'nycha_dev_')
    name = name.replace('if you live in nycha or are a section 8 voucher holder, please check all that apply below', 'nycha_sec8_')
    name = name.replace('are you a veteran of the military or military spouse or gold star family member?', 'veteran_')
    name = name.replace('have you come up with an idea for a business or venture that you would like to pursue?', 'idea_stage_')
    name = name.replace('what kind of product or service do you want to offer to people? please describe very briefly below.', 'product_desc_')
    name = name.replace('do you have a name for your organization or company?', 'org_name_')
    name = name.replace('is your business is focused on any of the following industries? please check all that apply_', 'industry_')
    name = name.replace('what type of organization do you want to / have started?', 'org_type_')
    name = name.replace('what is motivating you to start pursue business training or owning your own company? please check all that apply', 'motivation_')

    # THE BARRIERS FIX (Targets your most problematic column group)
    name = name.replace('do you have any major barriers to participating in entrepreneurship programs that you want to tell us about_', 'barriers_')

    # --- TARGETED ANSWER SHORTENING & CONJUNCTION CLEANUP ---
    name = name.replace('re_entering_the_workforce', 're_entry')
    name = name.replace('personal_credit_score', 'credit_score')
    name = name.replace('money_to_start_business', 'money_start')
    name = name.replace('_and_', '_').replace('_or_', '_').replace('_to_', '_').replace('_of_', '_').replace('_with_', '_').replace('_will_', '_')

    # --- FINAL UNIVERSAL CLEANUP ---
    name = re.sub(r'[^a-z0-9_]+', '_', name) # Replace any remaining punctuation/spaces with _
    name = re.sub(r'__+', '_', name)        # Collapse multiple underscores
    name = name.strip('_')                  # Remove leading/trailing underscores

    # --- THE CRITICAL TRUNCATION STEP (Guarantees short names) ---
    # Cut off any name over 60 characters (use 60 as it's safe and still readable)
    if len(name) > 60:
        # Keep the first 60 characters.
        name = name[:60]



    return name

df_user_featurescopy.columns = [final_guaranteed_shortener(col) for col in df_user_featurescopy.columns]

In [ ]:
df_user_featurescopy.head()

,email,timestamp,full_name,age_group,interest_entr_s,interest_social_s,interest_coop_s,borough_brooklyn,borough_manhattan,borough_queens,...,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren,do_you_have_any_major_barriers_participating_in_entrepren
0,ba...3@gmail.com,2025-03-15 16:16:26.778,Tanya B,49-64,1.000000,1.000000,0.00,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,kd...2@gmail.com,2025-03-15 16:16:45.279,Kd C,16-24,0.333333,0.000000,0.25,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,fl...t@verizon.net,2025-03-15 16:38:49.889,Lisa L,49-64,0.666667,0.666667,0.00,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,ja...2@gmail.com,2025-03-15 16:41:12.332,Jasiah V,16-24,0.666667,0.666667,0.75,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,dr...2@hotmail.com,2025-03-15 16:59:41.896,Renee B,34-48,1.000000,0.333333,0.50,1,0,0,...,1,0,0,0,0,0,0,0,0,0


Drop unnecessary columns

In [ ]:
feature_drop=['timestamp', 'full_name']
df_user_featurescopy.drop(columns= feature_drop, inplace= True)

print(df_user_featurescopy.columns.tolist())

['email', 'age_group', 'interest_entr_s', 'interest_social_s', 'interest_coop_s', 'borough_brooklyn', 'borough_manhattan', 'borough_queens', 'borough_the_bronx', 'housing_cityfheps_rent', 'housing_currently_staying_but_do_not_officially_live_in_nych', 'housing_housing', 'housing_i_am_a_section_8_voucher_holder', 'housing_i_don_t_live_in_nyhca_am_not_a_section_8_voucher', 'housing_i_live_in_nycha_public_housing', 'housing_outside_nycha', 'nycha_dev_abraham_lincoln_houses', 'nycha_dev_amsterdam_houses', 'nycha_dev_amsterdam_houses', 'nycha_dev_bronx_river_houses', 'nycha_dev_douglass_houses', 'nycha_dev_dyckman_housing_complex_3736_10th_avenue', 'nycha_dev_fulton_development', 'nycha_dev_fulton_houses', 'nycha_dev_gravesend_houses', 'nycha_dev_i_do_not_live_at_nycha', 'nycha_dev_millbrook_houses', 'nycha_dev_morrisania_air_rights', 'nycha_dev_morrisania_air_right', 'nycha_dev_mott_haven', 'nycha_dev_mott_haven', 'nycha_dev_mott_haven', 'nycha_dev_no', 'nycha_dev_patterson', 'nycha_dev_po

In [ ]:
df_survey_ID= df_sample_results[['Email']].drop_duplicates().rename(columns={'Email': 'Entrepreneur_Email'})  #removes duplicates and rename email column

# NLP Feature Engineering

In [ ]:
#reimport file package to prevent overwriting
from google.colab import files as NLP_files
import io
import pandas as pd

In [ ]:
uploaded = NLP_files.upload()
for NLP_files in uploaded.keys():
      df_NLP_results = pd.read_excel(io.BytesIO(uploaded[NLP_files]))
      print(f"DataFrame from {NLP_files}:")

Saving cleaned_survey_responses_data.xlsx to cleaned_survey_responses_data (3).xlsx
DataFrame from cleaned_survey_responses_data (3).xlsx:


In [ ]:
df_NLP_results.head()

,timestamp,email_address,full_name,age_group,nyc_borough,housing-status,nycha_housing_development_building,lease_owner_nycha,veteran_gold_star,interest_in_entrepreneurship,...,zip_code,income_level,social_welfare_public_housing,racial_ethnic,employment_status,self_employed_type,family_attended_college,closest_relative_attended_college,device_access,participation_barriers
0,2025-03-15 16:16:26.778,ba...3@gmail.com,Tanya B,49-64,Manhattan,I live in NYCHA public housing,Smith,My name is listed on the lease of my apartment...,No,5,...,10038,"Under $25,000",I am a resident of public or affordable housin...,Other: please describe,Self-Employed,FT,Yes,Parents & siblings,"Smartphone, Laptop",No
1,2025-03-15 16:16:45.279,kd...2@gmail.com,Kd C,16-24,the Bronx,Cityfheps/Rent,NaN,My name is listed on the lease of my apartment...,No,3,...,10458,"Under $25,000",None of these apply,African American / Black,Unemployed,NaN,Yes,My mother and she attended BMCC,"Smartphone, Laptop",No
2,2025-03-15 16:38:49.889,fl...t@verizon.net,Lisa L,49-64,Manhattan,I live in NYCHA public housing,Fulton Development,My name is listed on the lease of my apartment...,No,4,...,10011,"Under $25,000",I am a resident of public or affordable housin...,White,Unemployed,Unemployed,I'm not sure,NaN,Smartphone,No
3,2025-03-15 16:41:12.332,ja...2@gmail.com,Jasiah V,16-24,Manhattan,I live in NYCHA public housing,Smith houses,My name is listed on the lease of my apartment...,No,4,...,10048,"Under $25,000",I am a resident of public or affordable housin...,"African American / Black, Hispanic/Latinx, White",Employed,"Highline fellow civic engagement work, horticu...",Yes,Brother sister mom,Smartphone,NaN
4,2025-03-15 16:59:41.896,dr...2@hotmail.com,Renee B,34-48,Brooklyn,I am a Section 8 Voucher holder,NaN,My name is listed on the lease of my apartment...,No,5,...,11217,"Under $25,000",None of these apply,African American / Black,Employed,Part-time,Yes,Brother,"Smartphone, Computer",No


In [ ]:
df_user_idea = df_NLP_results[[
    'email_address',
    'product_service_description'
]].copy()

df_user_idea.rename(columns={'email_address': 'Entrepreneur_Email'}, inplace=True)

print("User Idea Text Table for NLP:")
print(df_user_idea.head())


print(f"\nTotal unique users for NLP: {df_user_idea['Entrepreneur_Email'].nunique()}")

User Idea Text Table for NLP:
   Entrepreneur_Email        product_service_description
0    ba...3@gmail.com                        Consulting 
1    kd...2@gmail.com                                NaN
2  fl...t@verizon.net     Hand cream and skin protector 
3    ja...2@gmail.com                                NaN
4  dr...2@hotmail.com  Art and chapbooks and stationary 

Total unique users for NLP: 35


Upload the other resources sheets to consolodate

In [ ]:

print("--- df_programs Columns ---")
print(df_programs.columns.tolist())

print("\n--- df_gen_resources Columns ---")
print(df_gen_resources.columns.tolist())

--- df_programs Columns ---
['program_name', 'eligibility_requirements', 'program_url']

--- df_gen_resources Columns ---
['resource_name', 'purpose', 'resource_url']


In [ ]:


# --- 1. Clean df_programs (Using 'eligibility_requirements' as description) ---
df_programs_clean = df_programs.rename(columns={
    'program_name': 'Resource_Name',
    'eligibility_requirements': 'Resource_Description'
})[['Resource_Name', 'Resource_Description']].copy()

# --- 2. Clean df_gen_resources (Using 'purpose' as description) ---
df_gen_resources_clean = df_gen_resources.rename(columns={
    'resource_name': 'Resource_Name',
    'purpose': 'Resource_Description'
})[['Resource_Name', 'Resource_Description']].copy()


# --- 3. Combine and Deduplicate ---
df_resource_text = pd.concat([df_programs_clean, df_gen_resources_clean], ignore_index=True)

# Remove duplicates (keeps the first description)
df_resource_text.drop_duplicates(subset=['Resource_Name'], keep='first', inplace=True)

# --- 4. Fill NaNs ---
# Fill any missing descriptions with an empty string so the vectorizer doesn't crash
df_resource_text['Resource_Description'] = df_resource_text['Resource_Description'].fillna('')

print("Resource Description Table for NLP created.")
print(df_resource_text.head())

Resource Description Table for NLP created.
                    Resource_Name  \
0                          Cambio   
1                   REES Dreamers   
2                  REES Idealists   
3  REES Underground Entrepreneurs   
4                 REES Go-getters   

                                Resource_Description  
0                  NYCHA resident, NOT Scaling Stage  
1                                         Idea Stage  
2  NOT Idea stage, Legally registered business, N...  
3  NOT Idea stage, NO Legally registered business...  
4  NOT Idea stage, legally registered business, l...  


In [ ]:
#copied from AI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load the skeleton created in your previous block
df_skeleton = pd.read_csv('df_training_set_skeleton.csv')

# --- Step A: Merge Text onto Skeleton ---
# df_user_idea and df_resource_text are assumed to be available
df_working = pd.merge(
    df_skeleton,
    df_user_idea, # This is the df_user_idea table
    on='Entrepreneur_Email',
    how='left'
)

df_working = pd.merge(
    df_working,
    df_resource_text,
    on='Resource_Name',
    how='left'
)

# --- Step B: Prepare Text for Vectorization ---
df_working['product_service_description'] = df_working['product_service_description'].fillna('')
df_working['Resource_Description'] = df_working['Resource_Description'].fillna('')
all_text = pd.concat([
    df_working['product_service_description'],
    df_working['Resource_Description']
])

# --- Step C: TF-IDF Vectorization ---
tfidf = TfidfVectorizer(stop_words='english', min_df=5)
tfidf_matrix = tfidf.fit_transform(all_text)
num_rows = len(df_working)
user_vectors = tfidf_matrix[:num_rows]
resource_vectors = tfidf_matrix[num_rows:]


# --- Step D: Calculate Cosine Similarity ---
similarity_scores = [
    cosine_similarity(user_vectors[i], resource_vectors[i])[0][0]
    for i in range(num_rows)
]

# --- Step E: Create the Final Feature Table ---
df_similarity_scores = df_working[[
    'Entrepreneur_Email',
    'Resource_Name'
]].copy()

df_similarity_scores['Idea_Resource_Similarity_Score'] = similarity_scores

df_similarity_scores.to_csv('df_similarity_scores.csv', index=False)




SUCCESS! The final NLP feature table has been created as 'df_similarity_scores.csv'


# Merging all the data

In [ ]:
# --- 1. Drop the text columns from the OHE features
columns_to_drop = [
    'product_desc_', # The raw text column we used for NLP
    'org_name_'      # The organization name (also text, not useful as OHE)
]

# Create the final OHE table
df_user_features_ohe = df_user_featurescopy.drop(columns=columns_to_drop, errors='ignore')

# 2. Rename the Email column for a clean merge
df_user_features_ohe.rename(columns={'email': 'Entrepreneur_Email'}, inplace=True)

# 3. Save the final OHE features to a CSV file
df_user_features_ohe.to_csv('df_user_features_ohe.csv', index=False)

print("OHE Features successfully saved as 'df_user_features_ohe.csv'")

OHE Features successfully saved as 'df_user_features_ohe.csv'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from google.colab import files # Import files for the download prompt

# Define the final Excel filename
final_excel_name = 'df_final_training_set_COMPLETE.xlsx'

# ---------------------- A. LOAD ALL FEATURE COMPONENTS ----------------------
# (Assuming the FileNotFoundError was resolved and files are loaded)
df_final_training_set = pd.read_csv('df_training_set_skeleton.csv')
df_similarity = pd.read_csv('df_similarity_scores.csv')
df_user_ohe = pd.read_csv('df_user_features_ohe.csv')

# ---------------------- B. PERFORM ALL FINAL MERGES ----------------------
# 1. Merge the NLP Similarity Score
df_final_training_set = pd.merge(
    df_final_training_set,
    df_similarity,
    on=['Entrepreneur_Email', 'Resource_Name'],
    how='left'
)
# 2. Merge the OHE User Features
df_final_training_set = pd.merge(
    df_final_training_set,
    df_user_ohe,
    on='Entrepreneur_Email',
    how='left'
)

# ---------------------- C. FEATURE SELECTION (Low Variance Filter) ----------------------


# 1. Store the essential columns (Keys, Target Y, and the critical NLP Score)
keys_and_y = df_final_training_set[['Entrepreneur_Email', 'Resource_Name', 'is_recommended', 'Idea_Resource_Similarity_Score']]

# 2. Drop the essential columns to leave only the OHE features for filtering
X_full = df_final_training_set.drop(
    columns=['Entrepreneur_Email', 'Resource_Name', 'is_recommended', 'Idea_Resource_Similarity_Score']
)

# 3. Isolate ONLY the numerical OHE columns (floats/ints) for filtering
X_ohe = X_full.select_dtypes(include=[np.number])

# 4. Apply the Variance Threshold
selector = VarianceThreshold(threshold=0.001)
X_filtered = selector.fit_transform(X_ohe)

# 5. Rebuild the filtered feature dataframe
selected_cols = X_ohe.columns[selector.get_support(indices=True)]
df_X_filtered = pd.DataFrame(X_filtered, columns=selected_cols)

df_final_complete = pd.concat([keys_and_y.reset_index(drop=True), df_X_filtered.reset_index(drop=True)], axis=1)


# ---------------------- D. FINAL SAVE AS EXCEL AND DOWNLOAD ----------------------

# Save the final cleaned DataFrame as an Excel file
df_final_complete.to_excel(final_excel_name, index=False)

# Download the file
files.download(final_excel_name)

print("\nTask 4 Complete! 🎉")
print("The final dataset is complete, filtered, and includes the 'Idea_Resource_Similarity_Score'.")
print(f"Final Training Dataset Shape: {df_final_complete.shape}")
print(f"Number of filtered OHE features: {len(selected_cols)}")
print(f"Final data saved and downloaded as '{final_excel_name}'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Task 4 Complete! 🎉
The final dataset is complete, filtered, and includes the 'Idea_Resource_Similarity_Score'.
Final Training Dataset Shape: (1089, 153)
Number of filtered OHE features: 149
Final data saved and downloaded as 'df_final_training_set_COMPLETE.xlsx'
